In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
! ls 'drive/MyDrive'

'20231421 (1).jpg'		      IMG_20231016_210131_375.jpg
 20231421.docx			      IMG_20240105_210703.jpg
 20231421.jpg			     'InnovateX university application form (1).gdoc'
 advertising.csv		     'InnovateX university application form.gdoc'
 AllPastStudywithModels.ipynb	     'InnovateX university application form.pdf'
'CM1607 Coursework'		      InteractiveSheet_2024-12-19_16_28_50.gsheet
 CM2604				      InteractiveSheet_2024-12-19_16_30_31.gsheet
 CM2607				      InteractiveSheet_2024-12-20_11_05_56.gsheet
'Codefest - AI Hackathon'	      InteractiveSheet_2024-12-20_11_23_09.gsheet
'Colab Notebooks'		      InteractiveSheet_2024-12-23_22_44_00.gsheet
 College_Data.csv		     'Lecture 1.mp4'
'Copy of Final.docx'		     'Lesson 5-SciKit Learn.ipynb'
 CV				      MortalityDATA.xlsx
'Deferral Request.docx'		     'photo_2023-09-21_07-43-40 (1).jpg'
 Diabetes_data.csv		      photo_2023-09-21_07-43-40.jpg
'Dilakshan Rahul.jpg'		      Screenshot_2024-01-05-21-05-36-062_lk.nsbpay.user.jpg
 DSGP				 

In [ ]:
dataset = 'drive/MyDrive/standardiazed_dataset_31feat.csv' # Change to dataset

In [ ]:
df = pd.read_csv(dataset)
df.shape

(15223, 33)

In [ ]:
X = df.copy()
y = X['hospital_death']  #target variable
X = X.drop(['hospital_death','Unnamed: 0'], axis=1)
X.shape

(15223, 31)

In [ ]:
X.columns

Index(['age', 'bmi', 'pre_icu_los_days', 'd1_diasbp_max', 'd1_diasbp_min',
       'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min',
       'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_min', 'd1_sysbp_max',
       'd1_sysbp_min', 'd1_temp_max', 'd1_temp_min', 'h1_diasbp_max',
       'h1_diasbp_min', 'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_max',
       'h1_mbp_min', 'h1_resprate_max', 'h1_resprate_min', 'h1_spo2_min',
       'h1_sysbp_max', 'h1_sysbp_min', 'd1_glucose_max', 'd1_glucose_min',
       'd1_potassium_max', 'd1_potassium_min'],
      dtype='object')

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

# Apply SMOTE to oversample the minority class in the training data
smote = SMOTE(random_state=11)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Initialize and train a RandomForestClassifier on the resampled data
rf_model_smote = RandomForestClassifier(n_estimators=100, random_state=2)
rf_model_smote.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=2)

In [ ]:
from sklearn.metrics import classification_report

# Make predictions on the test set
y_pred = rf_model_smote.predict(X_test)

# Generate and print the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2796
           1       0.34      0.31      0.32       249

    accuracy                           0.89      3045
   macro avg       0.64      0.63      0.63      3045
weighted avg       0.89      0.89      0.89      3045



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(100, 300),  # Number of trees in the forest
    'max_depth': list(range(50, 150, 10)),  # Maximum depth of the trees
    'min_samples_split': randint(2, 10),  # Minimum samples required to split an internal node
    'min_samples_leaf': randint(1, 10),  # Minimum samples required to be at a leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_model_smote,
    param_distributions=param_dist,
    n_iter=50,  # Number of random combinations to try
    cv=5,       # Number of cross-validation folds
    scoring='f1',  # Use F1-score for evaluation (adjust as needed)
    n_jobs=-1,     # Use all available CPU cores
    random_state=42,
    verbose=1  # Print progress
)


# Perform hyperparameter tuning
random_search.fit(X_train_resampled, y_train_resampled)

# Print the best hyperparameters and the corresponding score
print("Best Hyperparameters:", random_search.best_params_)
print("Best F1 Score:", random_search.best_score_)

# Use the best model for predictions
best_rf_model = random_search.best_estimator_
y_pred = best_rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters: {'bootstrap': False, 'max_depth': 130, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 147}
Best F1 Score: 0.9593803338845607
              precision    recall  f1-score   support

           0       0.93      0.96      0.95      2796
           1       0.37      0.23      0.28       249

    accuracy                           0.90      3045
   macro avg       0.65      0.60      0.62      3045
weighted avg       0.89      0.90      0.89      3045

